# Comments analysis

In [24]:
! pip install --upgrade pip
! pip install fasttext
! pip install emoji
! pip install pandas emoji openpyxl

In [25]:
import pandas as pd
import re
import fasttext
import emoji

In [26]:
# Defining file paths for Ukrainian and Russian channel comments
ukr_files = [
    "data/ua/Comments_ds.csv",
    "data/ua/Comments_ko.csv",
    "data/ua/Comments_uo.csv",
    "data/ua/Comments_no.csv"
]

ru_files = [
    "data/ru/Comments_cl.csv",
    "data/ru/Comments_dm.csv",
    "data/ru/Comments_re.csv"
]

# Helper function to load and label each DataFrame
def load_and_label_comments(file_list, channel_orientation_label):
    df_list = []
    for file in file_list:
        df = pd.read_csv(file)
        df["channel_orientation"] = channel_orientation_label
        df["source_file"] = file
        df_list.append(df)
    return pd.concat(df_list, ignore_index=True)

# Loading all Ukrainian and Russian comments
ukr_comments = load_and_label_comments(ukr_files, "pro-ukrainian")
ru_comments = load_and_label_comments(ru_files, "pro-russian")

# Combining both into a single DataFrame
df_all_comments = pd.concat([ukr_comments, ru_comments], ignore_index=True)

# Displaying basic info
df_all_comments

,post_id,comment_text,comment_date,comment_user_id,channel_orientation,source_file
0,21462,головне що кацапи дохнуть в муках. а ті кацапи...,2025-03-14 09:27:40+00:00,6.189523e+09,pro-ukrainian,data/ua/Comments_ds.csv
1,21462,у кожного може бути своя думка... без проблем,2025-03-14 06:58:47+00:00,6.763106e+09,pro-ukrainian,data/ua/Comments_ds.csv
2,21462,ну ти майже такий самий стратег як радіоактивн...,2025-03-14 06:57:49+00:00,8.454073e+08,pro-ukrainian,data/ua/Comments_ds.csv
3,21462,"так, саме так і думаю .",2025-03-14 06:56:22+00:00,6.763106e+09,pro-ukrainian,data/ua/Comments_ds.csv
4,21462,"Люди які кажуть, що росія зайшла на Сумщину ч...",2025-03-14 06:49:36+00:00,9.346062e+08,pro-ukrainian,data/ua/Comments_ds.csv
...,...,...,...,...,...,...
541662,93009,Черномазый сын шлюхи,2025-02-26 04:03:12+00:00,7.526166e+09,pro-russian,data/ru/Comments_re.csv
541663,93009,Муфтий от слова муфта,2025-02-21 04:45:54+00:00,1.076707e+09,pro-russian,data/ru/Comments_re.csv
541664,93009,Как они не поймут они потомки проигравших. Они...,2025-02-20 19:05:05+00:00,1.183544e+09,pro-russian,data/ru/Comments_re.csv
541665,93009,Это кто такой вообще?,2025-02-20 15:50:46+00:00,9.271602e+08,pro-russian,data/ru/Comments_re.csv


### Text cleaning

In [27]:
def clean_text(comment_text):
    if pd.isnull(comment_text):
        return ""
    comment_text = re.sub(r"http\S+|www\S+|t\.me\S+", "", comment_text)  # remove URLs
    comment_text = re.sub(r"@\w+", "", comment_text)                     # remove mentions
    comment_text = re.sub(r"#\w+", "", comment_text)                     # remove hashtags
    return comment_text

In [28]:
df_all_comments["comment_text"] = df_all_comments["comment_text"].apply(clean_text)

### Detecting language

In [29]:
# Loading the language detection model
lang_model = fasttext.load_model("data/lid.176.ftz")

# Defining language detection function
def detect_language(text):
    if not isinstance(text, str) or len(text.strip()) < 10:
        return "unknown"
    prediction = lang_model.predict(text.replace('\n', ' '), k=1)
    return prediction[0][0].replace("__label__", "")

In [30]:
df_all_comments = df_all_comments[df_all_comments["comment_text"].notna()]
df_all_comments["comment_language"] = df_all_comments["comment_text"].apply(detect_language)
df_all_comments

,post_id,comment_text,comment_date,comment_user_id,channel_orientation,source_file,comment_language
0,21462,головне що кацапи дохнуть в муках. а ті кацапи...,2025-03-14 09:27:40+00:00,6.189523e+09,pro-ukrainian,data/ua/Comments_ds.csv,uk
1,21462,у кожного може бути своя думка... без проблем,2025-03-14 06:58:47+00:00,6.763106e+09,pro-ukrainian,data/ua/Comments_ds.csv,uk
2,21462,ну ти майже такий самий стратег як радіоактивн...,2025-03-14 06:57:49+00:00,8.454073e+08,pro-ukrainian,data/ua/Comments_ds.csv,uk
3,21462,"так, саме так і думаю .",2025-03-14 06:56:22+00:00,6.763106e+09,pro-ukrainian,data/ua/Comments_ds.csv,uk
4,21462,"Люди які кажуть, що росія зайшла на Сумщину ч...",2025-03-14 06:49:36+00:00,9.346062e+08,pro-ukrainian,data/ua/Comments_ds.csv,uk
...,...,...,...,...,...,...,...
541662,93009,Черномазый сын шлюхи,2025-02-26 04:03:12+00:00,7.526166e+09,pro-russian,data/ru/Comments_re.csv,ru
541663,93009,Муфтий от слова муфта,2025-02-21 04:45:54+00:00,1.076707e+09,pro-russian,data/ru/Comments_re.csv,ru
541664,93009,Как они не поймут они потомки проигравших. Они...,2025-02-20 19:05:05+00:00,1.183544e+09,pro-russian,data/ru/Comments_re.csv,ru
541665,93009,Это кто такой вообще?,2025-02-20 15:50:46+00:00,9.271602e+08,pro-russian,data/ru/Comments_re.csv,ru


# Removing duplicates

In [31]:
df_all_comments = df_all_comments.drop_duplicates(subset="comment_text")

# Normalising whitespace

In [32]:
df_all_comments.loc["comment_text"] = df_all_comments["comment_text"].str.replace(r'\s+', ' ', regex=True).str.strip()

/var/folders/x9/4vdx7j915qg5pvqlbqynp2dr0000gn/T/ipykernel_10749/3833355000.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all_comments.loc["comment_text"] = df_all_comments["comment_text"].str.replace(r'\s+', ' ', regex=True).str.strip()


# Lowercasing

In [33]:
df_all_comments["comment_text"] = df_all_comments["comment_text"].str.lower()
df_all_comments

/var/folders/x9/4vdx7j915qg5pvqlbqynp2dr0000gn/T/ipykernel_10749/1695691445.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all_comments["comment_text"] = df_all_comments["comment_text"].str.lower()


,post_id,comment_text,comment_date,comment_user_id,channel_orientation,source_file,comment_language
0,21462.0,головне що кацапи дохнуть в муках. а ті кацапи...,2025-03-14 09:27:40+00:00,6.189523e+09,pro-ukrainian,data/ua/Comments_ds.csv,uk
1,21462.0,у кожного може бути своя думка... без проблем,2025-03-14 06:58:47+00:00,6.763106e+09,pro-ukrainian,data/ua/Comments_ds.csv,uk
2,21462.0,ну ти майже такий самий стратег як радіоактивн...,2025-03-14 06:57:49+00:00,8.454073e+08,pro-ukrainian,data/ua/Comments_ds.csv,uk
3,21462.0,"так, саме так і думаю .",2025-03-14 06:56:22+00:00,6.763106e+09,pro-ukrainian,data/ua/Comments_ds.csv,uk
4,21462.0,"люди які кажуть, що росія зайшла на сумщину ч...",2025-03-14 06:49:36+00:00,9.346062e+08,pro-ukrainian,data/ua/Comments_ds.csv,uk
...,...,...,...,...,...,...,...
541662,93009.0,черномазый сын шлюхи,2025-02-26 04:03:12+00:00,7.526166e+09,pro-russian,data/ru/Comments_re.csv,ru
541663,93009.0,муфтий от слова муфта,2025-02-21 04:45:54+00:00,1.076707e+09,pro-russian,data/ru/Comments_re.csv,ru
541664,93009.0,как они не поймут они потомки проигравших. они...,2025-02-20 19:05:05+00:00,1.183544e+09,pro-russian,data/ru/Comments_re.csv,ru
541666,93005.0,это что такое,2025-02-27 05:20:47+00:00,7.478964e+09,pro-russian,data/ru/Comments_re.csv,ru


# Transforming emotion into a text


In [34]:
# Function to transform emojis
def transform_emojis(text):
    if pd.isnull(text):
        return ""
    return emoji.demojize(text, delimiters=(":"," "))

In [35]:
# Applying transformation to the "text" column
df_all_comments["text_transformed"] = df_all_comments["comment_text"].apply(transform_emojis)
df_all_comments

/var/folders/x9/4vdx7j915qg5pvqlbqynp2dr0000gn/T/ipykernel_10749/2665326198.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all_comments["text_transformed"] = df_all_comments["comment_text"].apply(transform_emojis)


,post_id,comment_text,comment_date,comment_user_id,channel_orientation,source_file,comment_language,text_transformed
0,21462.0,головне що кацапи дохнуть в муках. а ті кацапи...,2025-03-14 09:27:40+00:00,6.189523e+09,pro-ukrainian,data/ua/Comments_ds.csv,uk,головне що кацапи дохнуть в муках. а ті кацапи...
1,21462.0,у кожного може бути своя думка... без проблем,2025-03-14 06:58:47+00:00,6.763106e+09,pro-ukrainian,data/ua/Comments_ds.csv,uk,у кожного може бути своя думка... без проблем
2,21462.0,ну ти майже такий самий стратег як радіоактивн...,2025-03-14 06:57:49+00:00,8.454073e+08,pro-ukrainian,data/ua/Comments_ds.csv,uk,ну ти майже такий самий стратег як радіоактивн...
3,21462.0,"так, саме так і думаю .",2025-03-14 06:56:22+00:00,6.763106e+09,pro-ukrainian,data/ua/Comments_ds.csv,uk,"так, саме так і думаю ."
4,21462.0,"люди які кажуть, що росія зайшла на сумщину ч...",2025-03-14 06:49:36+00:00,9.346062e+08,pro-ukrainian,data/ua/Comments_ds.csv,uk,"люди які кажуть, що росія зайшла на сумщину ч..."
...,...,...,...,...,...,...,...,...
541662,93009.0,черномазый сын шлюхи,2025-02-26 04:03:12+00:00,7.526166e+09,pro-russian,data/ru/Comments_re.csv,ru,черномазый сын шлюхи
541663,93009.0,муфтий от слова муфта,2025-02-21 04:45:54+00:00,1.076707e+09,pro-russian,data/ru/Comments_re.csv,ru,муфтий от слова муфта
541664,93009.0,как они не поймут они потомки проигравших. они...,2025-02-20 19:05:05+00:00,1.183544e+09,pro-russian,data/ru/Comments_re.csv,ru,как они не поймут они потомки проигравших. они...
541666,93005.0,это что такое,2025-02-27 05:20:47+00:00,7.478964e+09,pro-russian,data/ru/Comments_re.csv,ru,это что такое


# Merging posts with comments datasets

In [36]:
# Loading posts dataframe already with filtered relevant posts
df_posts = pd.read_csv("data/analysis/processed_csv/relevant_posts.csv")
df_posts

,post_id,date,text,views,forwards,channel_id,channel_name,language,date_only,text_transformed,...,dehumanization,mockery,credibility,USA_blame,Russia_blame,propaganda_technique,narrative_summary,source,bloc,period
0,156507,2025-03-04 08:47:39,"американские сми заявляют, что вчера венгрия з...",205470,292,-1001101806611,boris_rozhin,ru,2025-03-04,"американские сми заявляют, что вчера венгрия з...",...,1.0,1.0,NaN,1.0,0.0,"sarcasm, rhetorical ridicule, selective quotin...",Ukraine must grovel to Trump and the EU for su...,cl,pro-russian,post
1,156500,2025-03-04 07:28:05,бывший мэр нью-йорка и адвокат трампа рудольф ...,216760,642,-1001101806611,boris_rozhin,ru,2025-03-04,бывший мэр нью-йорка и адвокат трампа рудольф ...,...,1.0,1.0,1.0,0.0,0.0,"sarcasm, rhetorical ridicule, emotional appeal","Giuliani joined in denouncing Zelensky, and mo...",cl,pro-russian,post
2,156491,2025-03-04 03:15:49,**вице-президент сша вэнс:**\n\n1. украина не ...,238309,544,-1001101806611,boris_rozhin,ru,2025-03-04,**вице-президент сша вэнс:**\n\n1. украина не ...,...,0.0,0.0,1.0,0.0,0.0,"rhetorical assertion, simplification, attribut...",Vance said Ukraine is incapable of continuing ...,cl,pro-russian,post
3,156489,2025-03-04 02:47:15,"**зеленский укусил руку, которая его кормит, к...",223542,322,-1001101806611,boris_rozhin,ru,2025-03-04,"**зеленский укусил руку, которая его кормит, к...",...,0.0,0.0,0.0,0.0,0.0,"metaphor of subordination, emotional appeal, m...","Even though presented as a “quote,” no evidenc...",cl,pro-russian,post
4,156483,2025-03-04 00:38:57,**белый дом подтвердил приостановку военной по...,212596,612,-1001101806611,boris_rozhin,ru,2025-03-04,**белый дом подтвердил приостановку военной по...,...,0.0,1.0,1.0,0.0,0.0,"rhetorical intimidation, emotional appeal, moc...",The White House confirmed suspension of aid an...,cl,pro-russian,post
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,95885,2025-02-27 18:33:14,"**мої стосунки з зеленським стали ""дещо напруж...",225883,236,-1001233777422,UaOnlii,uk,2025-02-27,"**мої стосунки з зеленським стали ""дещо напруж...",...,0.0,1.0,2.0,1.0,0.0,"sarcasm, rhetorical question, emotional appeal",Sarcastically questions why relations are tens...,uo,pro-ukrainian,pre
318,95884,2025-02-27 18:28:36,"**❗️я не можу повірити, що назвав президента з...",229000,1562,-1001233777422,UaOnlii,uk,2025-02-27,"**:exclamation_mark я не можу повірити, що наз...",...,0.0,1.0,2.0,1.0,0.0,"sarcasm, rhetorical ridicule, emotional appeal",Mocks Trump’s denial by sarcastically claiming...,uo,pro-ukrainian,pre
319,95883,2025-02-27 18:22:07,"❗️**трамп на запитання, чи вважає він, як і ра...",226815,1520,-1001233777422,UaOnlii,uk,2025-02-27,":exclamation_mark **трамп на запитання, чи вва...",...,0.0,0.0,2.0,1.0,0.0,"rhetorical understatement, implied critique",Uses three moai emojis to signal skepticism an...,uo,pro-ukrainian,pre
320,95881,2025-02-27 18:10:26,"❗️**я не думаю, що путін знову вторгнеться, як...",218506,237,-1001233777422,UaOnlii,uk,2025-02-27,":exclamation_mark **я не думаю, що путін знову...",...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,uo,pro-ukrainian,pre


In [37]:
df_all_comments = df_all_comments.copy()

In [38]:
# Parsing timestamps
df_posts["date"] = pd.to_datetime(df_posts["date"], errors="coerce")
df_all_comments["comment_date"] = pd.to_datetime(df_all_comments["comment_date"], errors="coerce")

In [39]:
# Preparing post-level data (drop duplicates just in case)
keep_columns = [
    "post_id", "date", "views", "forwards", "channel_id",
    "channel_name", "source", "bloc", "period"
]
df_posts_clean = df_posts[keep_columns].drop_duplicates(subset=["post_id"])

In [40]:
print(df_all_comments["post_id"].dtype)
print(df_posts_clean["post_id"].dtype)

float64
int64


In [41]:
valid_post_ids = set(df_posts_clean["post_id"])
df_all_comments = df_all_comments[df_all_comments["post_id"].isin(valid_post_ids)]

In [42]:
# Merging post metadata into comments
df_all = df_all_comments.merge(df_posts_clean, on="post_id", how="left")

In [43]:
df_all[["post_id", "date", "comment_date"]].dropna().sample(5)

,post_id,date,comment_date
16933,53606.0,2025-02-28 17:26:37,2025-02-28 17:32:14+00:00
28038,93559.0,2025-02-28 20:36:52,2025-02-28 22:23:39+00:00
26241,93598.0,2025-03-01 19:34:28,2025-03-01 20:52:17+00:00
12606,95985.0,2025-02-28 18:00:15,2025-02-28 19:01:34+00:00
16443,53610.0,2025-02-28 18:27:10,2025-02-28 18:54:54+00:00


In [44]:
# Normalizing timezones (make both tz-naive)
try:
    df_all["comment_date"] = df_all["comment_date"].dt.tz_localize(None)
except Exception:
    pass

try:
    df_all["date"] = df_all["date"].dt.tz_localize(None)
except Exception:
    pass

# Compute time delta from post to comment (in hours)
df_all["post_age_at_comment"] = (
    df_all["comment_date"] - df_all["date"]
).dt.total_seconds() / 3600

In [45]:
df_all

,post_id,comment_text,comment_date,comment_user_id,channel_orientation,source_file,comment_language,text_transformed,date,views,forwards,channel_id,channel_name,source,bloc,period,post_age_at_comment
0,21390.0,"there will be no freedom, first trump is prepa...",2025-03-02 10:28:41,7.868621e+09,pro-ukrainian,data/ua/Comments_ds.csv,en,"there will be no freedom, first trump is prepa...",2025-02-28 20:20:56,441866,1146,-1001469021333,DeepStateUA,ds,pro-ukrainian,post,38.129167
1,21390.0,i think us and ukraine could have a very good ...,2025-03-02 10:27:22,8.170125e+09,pro-ukrainian,data/ua/Comments_ds.csv,en,i think us and ukraine could have a very good ...,2025-02-28 20:20:56,441866,1146,-1001469021333,DeepStateUA,ds,pro-ukrainian,post,38.107222
2,21390.0,ти антисеміт!!!ти гітлера підтримуєш???\nце як...,2025-03-02 10:10:15,7.868621e+09,pro-ukrainian,data/ua/Comments_ds.csv,uk,ти антисеміт!!!ти гітлера підтримуєш???\nце як...,2025-02-28 20:20:56,441866,1146,-1001469021333,DeepStateUA,ds,pro-ukrainian,post,37.821944
3,21390.0,i think same goes for israel. lots of american...,2025-03-02 10:09:37,8.170125e+09,pro-ukrainian,data/ua/Comments_ds.csv,en,i think same goes for israel. lots of american...,2025-02-28 20:20:56,441866,1146,-1001469021333,DeepStateUA,ds,pro-ukrainian,post,37.811389
4,21390.0,"тобто не посилати гроші в ізраїль, бо бідним а...",2025-03-02 10:08:18,7.868621e+09,pro-ukrainian,data/ua/Comments_ds.csv,uk,"тобто не посилати гроші в ізраїль, бо бідним а...",2025-02-28 20:20:56,441866,1146,-1001469021333,DeepStateUA,ds,pro-ukrainian,post,37.789444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32885,93453.0,земли - россии 🇷🇺\nдолги - европе 🇪🇺\nденьги -...,2025-02-26 15:41:55,5.057871e+08,pro-russian,data/ru/Comments_re.csv,ru,земли - россии :Russia \nдолги - европе :Europ...,2025-02-26 15:37:46,397626,1383,-1001260622817,readovkanews,re,pro-russian,pre,0.069167
32886,93453.0,быть друзьями россии и быть с россией на равны...,2025-02-26 15:41:24,5.203758e+09,pro-russian,data/ru/Comments_re.csv,ru,быть друзьями россии и быть с россией на равны...,2025-02-26 15:37:46,397626,1383,-1001260622817,readovkanews,re,pro-russian,pre,0.060556
32887,93453.0,"50%? америкосы заберут 150,в этом сомневаться ...",2025-02-26 15:39:35,5.169864e+09,pro-russian,data/ru/Comments_re.csv,ru,"50%? америкосы заберут 150,в этом сомневаться ...",2025-02-26 15:37:46,397626,1383,-1001260622817,readovkanews,re,pro-russian,pre,0.030278
32888,93453.0,доня агент кремля,2025-02-26 15:38:56,2.033804e+09,pro-russian,data/ru/Comments_re.csv,ru,доня агент кремля,2025-02-26 15:37:46,397626,1383,-1001260622817,readovkanews,re,pro-russian,pre,0.019444


post_age_at_comment is the number of hours between the time a post was published on a Telegram channel and the time a specific comment was made in response to that post.
It’s calculated as:
post_age_at_comment = comment_date - post_date

I calculated it because:
1. Not all comments are equally reactive. A comment written 3 minutes after a post is very different in intent and engagement than one left 3 days later.

 2. By computing the age of the post at the time of each comment, I now have a temporal signal — a way to measure when users engage, not just what they say.